ref: https://deepeval.com/guides/guides-ai-agent-evaluation
- 오픈 소스 LLM 평가 프레임워크
- Pytest처럼 LLM 출력에 대해 손쉽게 유닛 테스트를 작성할 수 있음
- 연구적으로 뒷받침된 것을 포함해, 30개 이상의 LLM 기반 평가 지표를 바로 가져다 쓸 수 있음
- E2E(end-to-end)와 컴포넌트 레벨 평가를 모두 지원
- RAG, 에이전트, 챗봇 등 사실상 대부분의 LLM 활용 사례에 대한 평가 지원
- 최신 evolution 기법을 활용한 합성(synthetic) 데이터셋 생성 기능
- 지표(metric)를 간단히 커스터마이즈할 수 있고, 다양한 활용 케이스를 포괄

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

`@observe` 데코레이터를 이용하여 trace 가능하다.

자동적으로 trace 대상 코드를 찾는다고한다.

Confident AI 통해서 모니터링 할 수 있다.

In [2]:
import json
from openai import OpenAI
from deepeval.tracing import observe
from deepeval.dataset import Golden, EvaluationDataset

client = OpenAI()
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_flights",
            "description": "Search for available flights between two cities",
            "parameters": {
                "type": "object",
                "properties": {
                    "origin": {"type": "string"},
                    "destination": {"type": "string"},
                    "date": {"type": "string"}
                },
                "required": ["origin", "destination", "date"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "book_flight",
            "description": "Book a specific flight by ID",
            "parameters": {
                "type": "object",
                "properties": {
                    "flight_id": {"type": "string"}
                },
                "required": ["flight_id"]
            }
        }
    }
]

@observe(type="tool")
def search_flights(origin, destination, date):
    # Simulated flight search
    return [{"id": "FL123", "price": 450}, {"id": "FL456", "price": 380}]

@observe(type="tool")
def book_flight(flight_id):
    # Simulated booking
    return {"confirmation": "CONF-789", "flight_id": flight_id}

@observe(type="llm")
def call_openai(messages):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools
    )
    return response

@observe(type="agent")
def travel_agent(user_input):
    messages = [{"role": "user", "content": user_input}]

    # LLM reasons about which tool to call
    response = call_openai(messages)
    tool_call = response.choices[0].message.tool_calls[0]
    args = json.loads(tool_call.function.arguments)

    # Execute the tool
    flights = search_flights(args["origin"], args["destination"], args["date"])

    # LLM decides to book the cheapest
    cheapest = min(flights, key=lambda x: x["price"])
    messages.append({"role": "assistant", "content": f"Found flights. Booking cheapest: {cheapest['id']}"})

    booking = book_flight(cheapest["id"])

    return f"Booked flight {cheapest['id']} for ${cheapest['price']}. Confirmation: {booking['confirmation']}"

In [3]:
from deepeval.metrics import PlanQualityMetric, PlanAdherenceMetric

plan_quality = PlanQualityMetric()
plan_adherence = PlanAdherenceMetric()

In [4]:
from deepeval.dataset import EvaluationDataset, Golden

# Create dataset
dataset = EvaluationDataset(goldens=[
    Golden(input="Book a flight from NYC to London for next Monday")
])

# Loop through dataset with metrics
for golden in dataset.evals_iterator(metrics=[plan_quality, plan_adherence]):
    travel_agent(golden.input)

Output()



Metrics Summary

  - ✅ Plan Quality (score: 1.0, threshold: 0.5, strict: False, evaluation model: gpt-4.1, reason: There are no plans to evaluate within the trace of your agent's execution. Please check if the agent's planning or reasoning or thinking is stored in the trace attributes., error: None)
  - ✅ Plan Adherence (score: 1.0, threshold: 0.5, strict: False, evaluation model: gpt-4.1, reason: There were no plans to evaluate within the trace of your agent's execution. Please check if the agent's planning or reasoning or thinking is stored in any one of the trace attributes., error: None)
  - ✅ Plan Quality (score: 1.0, threshold: 0.5, strict: False, evaluation model: gpt-4.1, reason: There are no plans to evaluate within the trace of your agent's execution. Please check if the agent's planning or reasoning or thinking is stored in the trace attributes., error: None)
  - ✅ Plan Adherence (score: 1.0, threshold: 0.5, strict: False, evaluation model: gpt-4.1, reason: There were no p

⚠ WARNING: No hyperparameters logged.
» ]8;id=588779;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 4.08s | token cost: 0.026632000000000003 USD)
» Test Results (1 total tests):
   » Pass Rate: 100.0% | Passed: 1 | Failed: 0

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.